In [1]:
import pandas as pd
amtrakHistory = pd.read_csv('amtrackdata.csv')

In [2]:
amtrakHistory['Schedule Arrival Time']

0          09:45
1          10:37
2          11:27
3          11:51
4          12:35
           ...  
4800236    19:00
4800237    19:18
4800238    19:50
4800239    20:43
4800240    21:16
Name: Schedule Arrival Time, Length: 4800241, dtype: object

In [3]:
import pandas as pd
from datetime import datetime, time


# Convert to Date and extract Day of Week and Weekday/Weekend
amtrakHistory['Date'] = pd.to_datetime(amtrakHistory[['Year', 'Month', 'Day']])
amtrakHistory['Day of Week'] = amtrakHistory['Date'].dt.dayofweek
amtrakHistory['Weekday/Weekend'] = amtrakHistory['Day of Week'].apply(lambda x: 'Weekday' if x < 5 else 'Weekend')

def str_to_time(value):
    # Check if the value is already a time object
    if isinstance(value, time):  # <-- This is the change
        return value
    # If it's a string, attempt the conversion
    elif isinstance(value, str):
        return datetime.strptime(value, "%H:%M").time()
    # For any other data type (like NaNs or unexpected values), return None
    else:
        return None


columns_to_convert = ['Schedule Departure Time', 'Actual Departure Time', 'Schedule Arrival Time', 'Actual Arrival Time']
for column in columns_to_convert:
    amtrakHistory[column] = amtrakHistory[column].apply(str_to_time)


# Status determination
def compare_times(row):
    scheduled_time, actual_time = None, None
    if row['Schedule Departure Time'] and row['Actual Departure Time']:
        scheduled_time = row['Schedule Departure Time']
        actual_time = row['Actual Departure Time']
    elif row['Schedule Arrival Time'] and row['Actual Arrival Time']:
        scheduled_time = row['Schedule Arrival Time']
        actual_time = row['Actual Arrival Time']
    else:
        return None

    if actual_time > scheduled_time:
        return "Late"
    elif actual_time < scheduled_time:
        return "Early"
    else:
        return "On Time"

amtrakHistory['Status'] = amtrakHistory.apply(compare_times, axis=1)

# Calculate Scheduled Arrival Hour and Arrival Delay
amtrakHistory['Scheduled Arrival Hour'] = amtrakHistory['Schedule Arrival Time'].apply(lambda x: x.hour if x else None)
amtrakHistory['Arrival Delay (min)'] = amtrakHistory.apply(lambda row: (datetime.combine(datetime.today(), row['Actual Arrival Time']) - datetime.combine(datetime.today(), row['Schedule Arrival Time'])).total_seconds() / 60 if row['Actual Arrival Time'] and row['Schedule Arrival Time'] else None, axis=1)
amtrakHistory['Is Peak Hour'] = amtrakHistory['Scheduled Arrival Hour'].apply(lambda x: 1 if (7 <= x <= 9) or (16 <= x <= 18) else 0)

# Determine season
def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Autumn'
    else:
        return 'Winter'

amtrakHistory['Season'] = amtrakHistory['Month'].apply(get_season)


# Calculate delay and rolling average delay
amtrakHistory['Delay'] = amtrakHistory['Arrival Delay (min)']
N = 30
amtrakHistory['Rolling Avg Delay'] = amtrakHistory['Delay'].rolling(window=N).mean()


In [4]:
amtrakHistory["Weekday/Weekend"]

0          Weekday
1          Weekday
2          Weekday
3          Weekday
4          Weekday
            ...   
4800236    Weekend
4800237    Weekend
4800238    Weekend
4800239    Weekend
4800240    Weekend
Name: Weekday/Weekend, Length: 4800241, dtype: object

In [5]:
amtrakHistory["Weekday/Weekend"]

0          Weekday
1          Weekday
2          Weekday
3          Weekday
4          Weekday
            ...   
4800236    Weekend
4800237    Weekend
4800238    Weekend
4800239    Weekend
4800240    Weekend
Name: Weekday/Weekend, Length: 4800241, dtype: object

In [6]:
amtrakHistory['Status'].fillna('Unknown', inplace=True)


In [7]:
amtrakHistory.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)

In [8]:
import datetime

# ... rest of the code ...

def convert_to_time(x):
    if isinstance(x, datetime.time):
        return x
    else:
        try:
            return datetime.datetime.strptime(str(x), "%H:%M").time()
        except:
            return None


# Convert to Date and extract Day of Week and Weekday/Weekend
amtrakHistory['Date'] = pd.to_datetime(amtrakHistory[['Year', 'Month', 'Day']])
amtrakHistory['Day of Week'] = amtrakHistory['Date'].dt.dayofweek
amtrakHistory['Weekday/Weekend'] = amtrakHistory['Day of Week'].apply(lambda x: 'Weekday' if x < 5 else 'Weekend')

# Convert to time
amtrakHistory['Schedule Departure Time'] = amtrakHistory['Schedule Departure Time'].apply(convert_to_time)
amtrakHistory['Schedule Arrival Time'] = amtrakHistory['Schedule Arrival Time'].apply(convert_to_time)
amtrakHistory['Actual Departure Time'] = amtrakHistory['Actual Departure Time'].apply(convert_to_time)
amtrakHistory['Actual Arrival Time'] = amtrakHistory['Actual Arrival Time'].apply(convert_to_time)

# Calculate Duration of Stop

In [9]:
amtrakHistory

,Train ID,Year,Month,Day,Station Code,Schedule Arrival Day,Schedule Arrival Time,Schedule Departure Day,Schedule Departure Time,Actual Arrival Time,...,Date,Day of Week,Weekday/Weekend,Status,Scheduled Arrival Hour,Arrival Delay (min),Is Peak Hour,Season,Delay,Rolling Avg Delay
0,11,2019,1,1,SEA,*,09:45:00,1,09:45:00,09:45:00,...,2019-01-01,1,Weekday,On Time,9.0,0.0,1,Winter,0.0,NaN
1,11,2019,1,1,TAC,*,10:37:00,1,10:37:00,10:37:00,...,2019-01-01,1,Weekday,On Time,10.0,0.0,0,Winter,0.0,NaN
2,11,2019,1,1,OLW,*,11:27:00,1,11:27:00,11:27:00,...,2019-01-01,1,Weekday,On Time,11.0,0.0,0,Winter,0.0,NaN
3,11,2019,1,1,CTL,*,11:51:00,1,11:51:00,11:51:00,...,2019-01-01,1,Weekday,On Time,11.0,0.0,0,Winter,0.0,NaN
4,11,2019,1,1,KEL,*,12:35:00,1,12:35:00,12:38:00,...,2019-01-01,1,Weekday,Late,12.0,3.0,0,Winter,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4800236,99,2022,12,31,ASD,*,19:00:00,1,19:00:00,19:44:00,...,2022-12-31,5,Weekend,Late,19.0,44.0,0,Winter,44.0,21.333333
4800237,99,2022,12,31,RVR,1,19:18:00,1,19:23:00,19:58:00,...,2022-12-31,5,Weekend,Late,19.0,40.0,0,Winter,40.0,23.266667
4800238,99,2022,12,31,RVM,*,19:50:00,1,19:50:00,20:32:00,...,2022-12-31,5,Weekend,Late,19.0,42.0,0,Winter,42.0,25.233333
4800239,99,2022,12,31,WBG,*,20:43:00,1,20:43:00,21:26:00,...,2022-12-31,5,Weekend,Late,20.0,43.0,0,Winter,43.0,27.433333


In [10]:
import pandas as pd

holidays = {
    'New Year\'s Day': [(1, 1, year) for year in range(2019, 2023)],
    'Martin Luther King Jr. Day': [(1, 21, 2019), (1, 20, 2020), (1, 18, 2021), (1, 17, 2022)],
    'Presidents Day': [(2, 18, 2019), (2, 17, 2020), (2, 15, 2021), (2, 21, 2022)],
    'Memorial Day': [(5, 27, 2019), (5, 25, 2020), (5, 31, 2021), (5, 30, 2022)],
    'Independence Day': [(7, 4, year) for year in range(2019, 2023)],
    'Labor Day': [(9, 2, 2019), (9, 7, 2020), (9, 6, 2021), (9, 5, 2022)],
    'Columbus Day': [(10, 14, 2019), (10, 12, 2020), (10, 11, 2021), (10, 10, 2022)],
    'Veterans Day': [(11, 11, year) for year in range(2019, 2023)],
    'Thanksgiving Day': [(11, 28, 2019), (11, 26, 2020), (11, 25, 2021), (11, 24, 2022)],
    'Christmas Day': [(12, 25, year) for year in range(2019, 2023)]
}

flat_holidays = [date for sublist in holidays.values() for date in sublist]

def is_holiday(row):
    date_tuple = (row['Month'], row['Day'], row['Year'])
    return 1 if date_tuple in flat_holidays else 0

amtrakHistory['Is Holiday'] = amtrakHistory.apply(is_holiday, axis=1)

print(amtrakHistory[['Month', 'Day', 'Year', 'Is Holiday']].head())


   Month  Day  Year  Is Holiday
0      1    1  2019           1
1      1    1  2019           1
2      1    1  2019           1
3      1    1  2019           1
4      1    1  2019           1


In [11]:
amtrakHistory['ZIP'] = amtrakHistory['ZIP'].astype(str).str.slice(0, 5)
